# Notebook Bag of words
Jorge Antonio Magdaleno Rodriguez

In [ ]:
!pip install spacy
!pip install nltk
!pip install scikit-learn

In [ ]:
import pandas as pd
import re
import spacy as spc
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import re
import unicodedata

In [3]:
df = pd.read_csv('df_mini_HS.csv')
print(df.head())

   label                                               text
0      1  ESAS COSAS Y OTRAS PUEDEN PASAR POR MANTENER A...
1      1  28: te amodio, odio a la perra de tu amiga ☺️☺...
2      1  @LaDivinaDiva Callate maldita perra. O seguro ...
3      1  @MarysabelPuerto Mejor callate cara de puta o ...
4      1  @xarita327 @TRIKYHUMOR @yonier2012 @casTa1326 ...


Limpiamos los datos

In [4]:
df['text'] = df['text'].str.lower()

def remove_accents(oracion):
    oracion = unicodedata.normalize('NFD', oracion)
    oracion = ''.join(char for char in oracion if unicodedata.category(char) != 'Mn')
    return oracion

def clean_text(oracion):
    oracion = re.sub(r"@\S+", "", oracion)  # Eliminar menciones a usuarios
    oracion = re.sub(r"http[s]?\://\S+", "", oracion)  # Eliminar enlaces
    oracion = re.sub(r"#\S+", "", oracion)  # Eliminar hashtags
    oracion = re.sub(r"[0-9]", "", oracion)  # Eliminar números
    oracion = re.sub(r"(\(.*\))|(\[.*\])", "", oracion)  # Eliminar paréntesis y corchetes
    oracion = re.sub(r"\n", "", oracion)  # Eliminar caracteres de nueva línea
    oracion = re.sub(r"(http[s]?\://\S+)|([\[\(].*[\)\]])|([#@]\S+)|\n", "", oracion)  # Eliminar varios patrones
    oracion = re.sub(r"(\.)|(,)", "", oracion)  # Eliminar puntos y comas
    oracion = re.sub(r"[¡!]", "", oracion)  # Eliminar signos de admiración 
    oracion = re.sub(r"[¿?]", "", oracion)  # Eliminar signos de interrogación
    oracion = re.sub(r"[:]", "", oracion)  # Eliminar dos puntos
    oracion = re.sub(r"[\"]", "", oracion)  # Eliminar dos comillas
    oracion = re.sub(r"[\/]", "", oracion)  # Eliminar diagonales
    oracion = re.sub(r"[*]", "", oracion)  # Eliminar asteriscos
    oracion = re.sub(r"["
                     u"\U0001F600-\U0001F64F"  # Emoticonos
                     u"\U0001F300-\U0001F5FF"  # Símbolos y pictogramas
                     u"\U0001F680-\U0001F6FF"  # Transporte y símbolos de mapas
                     u"\U0001F1E0-\U0001F1FF"  # Banderas
                     u"\U00002702-\U000027B0"  # Otros símbolos
                     u"\U000024C2-\U0001F251"  # Más símbolos
                     "]+", "", flags=re.UNICODE, string=oracion)
    oracion = remove_accents(oracion)
    return oracion

df['text'] = df['text'].apply(clean_text)
print(df.iloc[1])

label                                                    1
text      te amodio odio a la perra de tu amiga pero tu...
Name: 1, dtype: object


Los tokenizamos y filtramos los stopwords. Los guardamos dentro del mismo df

In [5]:
#nltk.data.path.append('/Users/jorgemagdaleno/nltk_data')
#nltk.download('punkt')
#nltk.download('punkt_tab')

def remove_stopwords(oracion):
    tokens = word_tokenize(oracion)
    spanish_stopwords = stopwords.words('spanish')
    english_stopwords = stopwords.words('english')
    combined_stopwords = set(spanish_stopwords + english_stopwords)
    palabras_filtradas = [palabra for palabra in tokens if palabra.lower() not in combined_stopwords]
    return palabras_filtradas  # Return as a list

df['filtered_text'] = df['text'].apply(remove_stopwords)

print(df['filtered_text'])

0     [cosas, pueden, pasar, mantener, inmigracion, ...
1          [amodio, odio, perra, amiga, vida, tijerazo]
2     [callate, maldita, perra, seguro, pobre, maric...
3     [mejor, callate, cara, puta, reputa, madre, no...
4                                       [callate, puta]
5                                       [callate, puta]
6     [inmigrante, recibe, ayuda, rico, ladron, olvi...
7     [moros, puede, esperar, bueno, dicen, propios,...
8     [si, mujer, pegan, tiro, cabeza, dura, tres, d...
9     [analicemos, si, pones, shorts, asi, calle, es...
10    [see, tal, vez, recordo, peron, protegio, eich...
11    [pietrapierce, story, purs, sangs, arabes, sta...
12    [dice, frivolizar, acoso, escolar, favor, quer...
13    [retira, permiso, refugiados, vacaciones, pais...
14    [hoy, quiero, denunciaaaaaaar, gente, puto, gu...
15    [redomicilie, sociedad, offshore, emiratos, ar...
16    [basta, poned, pie, pared, tanta, provocacion,...
17    [semana, juventud, torneo, futbol, futbol,

Hacemos la lematizacion

In [6]:
nlp = spc.load("es_core_news_sm")

def lemmatize_text(filtered_text):
    lema_p = nlp(" ".join(filtered_text))
    oracion_lematizada_p = " ".join([token.lemma_ for token in lema_p])
    return oracion_lematizada_p

df['lemmatized_text'] = df['filtered_text'].apply(lemmatize_text)
print(df['lemmatized_text'])

0     cosa poder pasar mantener inmigracion ilegal e...
1                 amodio odio perra amigo vida tijerazo
2     callate maldito perra seguro pobre marico detr...
3     mejor callate cara puta reputa madre nota hace...
4                                          callate puto
5                                          callate puto
6     inmigrante recibir ayuda rico ladron olvidar n...
7     moro poder esperar bueno decir propio arab tam...
8     si mujer peguir tiro cabeza durar tres dia mor...
9     analicer si pón shorts asi calle esperas decir...
10    see tal vez recordo peron protegio eichmann ci...
11    pietrapiercir story purs sang arab stars des f...
12    decir frivolizar acoso escolar favor quereis d...
13          retirar permiso refugiado vacación país via
14    hoy quiero denunciaaaaaaar gente puto guarra o...
15     redomicilie sociedad offshore emirato arab unido
16    bastar poned pie pared tanto provocacion corta...
17    semana juventud torneo futbol futbol categ

Vectorizamos y guardamos dentro de df_vectorized, mostramos los valores del bag of words de la segunda frase

In [7]:
def vectorize_column(df, column_name):
    vectorizador = CountVectorizer()
    vectores = vectorizador.fit_transform(df[column_name])
    vectorized_df = pd.DataFrame(vectores.toarray(), columns=vectorizador.get_feature_names_out())
    #df = pd.concat([df, vectorized_df], axis=1)
    return vectorized_df
    
df_vectorized = vectorize_column(df, 'lemmatized_text')

print("Oración de entrada:", df['text'].loc[1])
print("Oración lematizada:", df['lemmatized_text'].loc[1])
print("Vectores Bag of Words:", df_vectorized.loc[1][df_vectorized.loc[1] != 0].values)
print("Vocabulario:", df_vectorized.loc[1][df_vectorized.loc[1] != 0].index)


Oración de entrada:  te amodio odio a la perra de tu amiga pero tu eres mi vida tijerazo
Oración lematizada: amodio odio perra amigo vida tijerazo
Vectores Bag of Words: [1 1 1 1 1 1]
Vocabulario: Index(['amigo', 'amodio', 'odio', 'perra', 'tijerazo', 'vida'], dtype='object')


Finalmente agregamos la clase basado en el label y tenemos nuestro bag of words

In [8]:
def assign_clase(label):
    return 'Misogino' if label == 1 else 'otro'

df_vectorized['clase'] = df['label'].apply(assign_clase)
df_vectorized = df_vectorized[['clase'] + [col for col in df_vectorized.columns if col != 'clase']]

df_vectorized.head()

,clase,acoso,agar,agresion,amigo,amodio,analicer,aprieto,arab,arar,...,user,vacación,verdad,vez,via,vida,viola,voolka,yogurín,él
0,Misogino,0,0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1,Misogino,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,Misogino,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,Misogino,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Misogino,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Como conclusión, utilizar palabras y frases reales es complicado debido a que existen muchas faltas de ortografía, el uso de emojis, palabras que solo se usan en grupos pequeños y varios otros problemas. Es necesario encontrar formas que nos permitan normalizar nuestros datos a un formato que la computadora pueda realmente entender.

Además, a lo largo de este proceso, hemos visto cómo aplicar técnicas de procesamiento de lenguaje natural como la tokenización, la eliminación de stopwords, la lematización y la vectorización, lo que nos ayuda a limpiar y transformar el texto en una representación más estructurada. Estos pasos son esenciales para mejorar la calidad de los datos textuales y, finalmente, hacerlos útiles para tareas de análisis más avanzadas o modelos de aprendizaje automático.